# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-daa4ae72bf-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the common aspects between movies directed by Woody Allen or Quentin Tarantino. We want to know the people that worked for both directors with some numerical analyses, what are the differences in terms of budget for their movies, who won more Academy Awards. 

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q2526255`| Director       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`    | Quentin Tarantino      | node |





Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**


The workload should:


1. Identify the BGP for films

2. Identify the BGP for directors

3. Identify the BGP for workers in a films

4. Compare the workers amongst the films directed by the two directors

5. Return some numerical comparison between the two directors (e.g., how many workers in Tarantino's movies who also worked in Allen's films?, what is the film with the highest number of shared actors? Who is the most used actor by both the directors? etc. )

6. Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?

7. Who has films with more nominations for Academy Awards and who won more Academy Awards (with his films not only personal awards).

    7.1 Find the BGP for Academy Awards 

    7.2 Find the related subproperties

    7.3 Find how they are related to the directors
    
    7.4 Are there alternative queries to get the same result?



In [2]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

wd:Q25089 ?p ?obj .
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '334')]


1

## Task 1 
Identify the BGP for films

Starting from Woody Allen, look to his property with a regex to find something like 'director'

In [6]:
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 

 wd:Q25089 ?pr ?o.
?pr <http://schema.org/name> ?name .
FILTER(REGEX(?name,\".*irect.*\"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1969'), ('name', 'MovieMeter director ID (former scheme)')]
[('pr', 'http://www.wikidata.org/prop/direct/P4283'), ('name', 'LUMIERE director ID')]


2

In [7]:
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 

?o ?pr wd:Q25089.
?pr <http://schema.org/name> ?name .
FILTER(REGEX(?name,\".*irect.*\"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P57'), ('name', 'director')]


1

Look for the class of the objects related with the predicate discover in the cell above

In [8]:
queryString = """
SELECT DISTINCT ?ist ?ista
WHERE { 

?o wdt:P57 wd:Q25089;
    wdt:P31 ?ist.
    
?ist <http://schema.org/name> ?ista .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('ist', 'http://www.wikidata.org/entity/Q11424'), ('ista', 'film')]
[('ist', 'http://www.wikidata.org/entity/Q506240'), ('ista', 'television film')]
[('ist', 'http://www.wikidata.org/entity/Q5398426'), ('ista', 'television series')]
[('ist', 'http://www.wikidata.org/entity/Q24862'), ('ista', 'short film')]
[('ist', 'http://www.wikidata.org/entity/Q47467768'), ('ista', 'operatic production')]


5

So, Q11424 identify a film

## Task 2
Identify the BGP for directors

Look at the subclass of human that is similar to director

In [9]:
queryString = """
SELECT DISTINCT ?ist ?ista
WHERE { 

?t wdt:P31/wdt:P279* wd:Q5;
    wdt:P31 ?ist.
?ist <http://schema.org/name> ?ista .
FILTER(REGEX(?ista,\".*irect.*\"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('ist', 'http://www.wikidata.org/entity/Q3455803'), ('ista', 'director')]
[('ist', 'http://www.wikidata.org/entity/Q56447510'), ('ista', 'Director')]
[('ist', 'http://www.wikidata.org/entity/Q1162163'), ('ista', 'director')]
[('ist', 'http://www.wikidata.org/entity/Q23931359'), ('ista', 'Director of Bureau')]


4

Look for the subclass of Director (Q56447510)

In [10]:
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?pr wdt:P279* wd:Q56447510.
?pr <http://schema.org/name> ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/entity/Q56447510'), ('name', 'Director')]


1

Look for the subclass of director (Q3455803)

In [11]:
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?pr wdt:P279* wd:Q3455803.
?pr <http://schema.org/name> ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/entity/Q27890911'), ('name', 'second unit director')]
[('pr', 'http://www.wikidata.org/entity/Q7579842'), ('name', 'sports director')]
[('pr', 'http://www.wikidata.org/entity/Q55092822'), ('name', 'ballet teacher')]
[('pr', 'http://www.wikidata.org/entity/Q61817103'), ('name', 'Director-producer')]
[('pr', 'http://www.wikidata.org/entity/Q2962070'), ('name', 'production designer')]
[('pr', 'http://www.wikidata.org/entity/Q17456089'), ('name', 'pornographic film director')]
[('pr', 'http://www.wikidata.org/entity/Q28054786'), ('name', 'opera director')]
[('pr', 'http://www.wikidata.org/entity/Q63538345'), ('name', 'video game director')]
[('pr', 'http://www.wikidata.org/entity/Q24204895'), ('name', 'art director')]
[('pr', 'http://www.wikidata.org/entity/Q28860513'), ('name', 'artistic coordinator')]
[('pr', 'http://www.wikidata.org/entity/Q1414443'), ('name', 'filmmaker')]
[('pr', 'http://www.wikidata.org/entity/Q2059704'), ('name', 'television 

56

This class seems to be the once that is related to a director of a movie <br>
Since I know that Q11424 is the class of film I will look if the film is related with this Q3455803.

In [12]:
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?o wdt:P31 wd:Q11424;
    wdt:P57 ?x.
?x wdt:P31/wdt:P279* ?pr.
?pr <http://schema.org/name> ?name .
FILTER(REGEX(?name,\".*irect.*\"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/entity/Q2526255'), ('name', 'film director')]
[('pr', 'http://www.wikidata.org/entity/Q3455803'), ('name', 'director')]


2

In fact I have a correspondence with Q3455803. So this is a generic director.<br>
I think that Q2526255 is a film director (specifically film, e.g. not tv series)

## Task 3
Identify the BGP for workers in a films

Look for some classes similar to 'worker' related with films starting from the class director that is Q3455803

In [24]:
queryString = """
SELECT DISTINCT ?sup ?name
WHERE { 
?o wdt:P31 wd:Q11424.
wd:Q3455803 wdt:P279* ?sup.
?obj wdt:P31 ?sup.
?obj ?pr ?o.
?sup <http://schema.org/name> ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sup', 'http://www.wikidata.org/entity/Q215627'), ('name', 'person')]
[('sup', 'http://www.wikidata.org/entity/Q43229'), ('name', 'organization')]
[('sup', 'http://www.wikidata.org/entity/Q16334295'), ('name', 'group of humans')]


3

Found nothing useful. Try to look for predicate from film to objects that are instance of human

In [3]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE { 
?o wdt:P31 wd:Q11424;
    ?p ?x.
?x wdt:P31 wd:Q5.
?p <http://schema.org/name> ?name .
} 
ORDER BY ?p
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1029'), ('name', 'crew member(s)')]
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('name', 'director / manager')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P110'), ('name', 'illustrator')]
[('p', 'http://www.wikidata.org/prop/direct/P123'), ('name', 'publisher')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('name', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1431'), ('name', 'executive producer')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based

79

It seems that all these predicate are related to someone that work on a film, let's look in the other direction

In [4]:
queryString = """
SELECT DISTINCT ?sup ?name
WHERE { 
?o wdt:P31 wd:Q11424.
?x ?sup ?o.
?x wdt:P31 wd:Q5.
?sup <http://schema.org/name> ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sup', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('sup', 'http://www.wikidata.org/prop/direct/P1299'), ('name', 'depicted by')]
[('sup', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('sup', 'http://www.wikidata.org/prop/direct/P1455'), ('name', 'list of works')]
[('sup', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')]
[('sup', 'http://www.wikidata.org/prop/direct/P3919'), ('name', 'contributed to creative work')]
[('sup', 'http://www.wikidata.org/prop/direct/P1441'), ('name', 'present in work')]
[('sup', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('sup', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('sup', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('sup', 'http://www.wikidata.org/prop/direct/P805'), ('name', 'statement is subject of')]
[('sup', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of 

73

In [22]:
queryString = """
SELECT DISTINCT ?sup ?name
WHERE { 
?o wdt:P31 wd:Q11424.
?x ?sup ?o.
?x wdt:P31 wd:Q5.
?sup <http://schema.org/name> ?name .
FILTER (REGEX(?name,".*ork.*"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sup', 'http://www.wikidata.org/prop/direct/P1455'), ('name', 'list of works')]
[('sup', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')]
[('sup', 'http://www.wikidata.org/prop/direct/P3919'), ('name', 'contributed to creative work')]
[('sup', 'http://www.wikidata.org/prop/direct/P1441'), ('name', 'present in work')]
[('sup', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('sup', 'http://www.wikidata.org/prop/direct/P767'), ('name', 'contributor to the creative work or subject')]
[('sup', 'http://www.wikidata.org/prop/direct/P1877'), ('name', 'after a work by')]
[('sup', 'http://www.wikidata.org/prop/direct/P1686'), ('name', 'for work')]
[('sup', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work')]


9

In [23]:
queryString = """
SELECT DISTINCT ?sup ?name
WHERE { 
?o wdt:P31 wd:Q11424.
?o ?sup ?x.
?x wdt:P31 wd:Q5.
?sup <http://schema.org/name> ?name .
FILTER (REGEX(?name,".*ork.*"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sup', 'http://www.wikidata.org/prop/direct/P1877'), ('name', 'after a work by')]
[('sup', 'http://www.wikidata.org/prop/direct/P767'), ('name', 'contributor to the creative work or subject')]


2

In [24]:
#use P710 participant
queryString = """
SELECT DISTINCT ?film ?name
WHERE { 

?o wdt:P31 wd:Q11424.
?x wdt:P710 ?o.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
?x <http://schema.org/name> ?name .
} 

LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('film', 'Marianne'), ('name', 'Georges Lourau')]
[('film', 'Madelon'), ('name', 'Georges Lourau')]
[('film', 'Mare matto'), ('name', 'Georges Lourau')]
[('film', 'Wild Fruit'), ('name', 'Georges Lourau')]
[('film', 'Holiday for Henrietta'), ('name', 'Georges Lourau')]
[('film', 'Madame du Barry'), ('name', 'Georges Lourau')]
[('film', 'Une souris chez les hommes'), ('name', 'Georges Lourau')]
[('film', 'Love in the Night'), ('name', 'Georges Lourau')]
[('film', 'The Professional'), ('name', 'Sidiki Bakaba')]


9

Try to use some predicate found above related with work:
- P800: notable work;
- P3919: contributed to creative work;
- P1441: present in work;
- P767: contributor to the creative work or subject;
- P1686: for work.

In [26]:
queryString = """
SELECT DISTINCT ?film (COUNT(DISTINCT ?x) AS ?worker)
WHERE { 
?o wdt:P31 wd:Q11424.
?x wdt:P800 ?o.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?worker)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'The Juniors y la Formula Imperial'), ('worker', '17')]
[('film', 'Evangelion: 1.0 You Are (Not) Alone'), ('worker', '12')]
[('film', 'Niños Asesinos Película'), ('worker', '11')]
[('film', 'El Triunfo de Vivir'), ('worker', '8')]
[('film', 'Guardians of the Galaxy'), ('worker', '8')]
[('film', 'Frozen'), ('worker', '8')]
[('film', 'Evangelion: 3.0+1.0 Thrice Upon a Time'), ('worker', '8')]
[('film', 'Aatsinki: The Story of Arctic Cowboys'), ('worker', '7')]
[('film', 'South of 8'), ('worker', '6')]
[('film', 'Liberation'), ('worker', '6')]


10

In [27]:
queryString = """
SELECT DISTINCT ?film (COUNT(DISTINCT ?x) AS ?worker)
WHERE { 
?o wdt:P31 wd:Q11424.
?x wdt:P3919 ?o.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?worker)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'The Great Hack'), ('worker', '4')]
[('film', 'Citizenfour'), ('worker', '1')]
[('film', 'Colonel Chabert'), ('worker', '1')]
[('film', 'The Longest Day'), ('worker', '1')]
[('film', 'Octopusalarm'), ('worker', '1')]
[('film', 'Mary Queen of Scots'), ('worker', '1')]
[('film', 'Night of the Seagulls'), ('worker', '1')]
[('film', "Aurelie Laflamme's Diary"), ('worker', '1')]
[('film', 'In the Waves'), ('worker', '1')]
[('film', 'Life of Pi'), ('worker', '1')]


10

In [28]:
queryString = """
SELECT DISTINCT ?film (COUNT(DISTINCT ?x) AS ?worker)
WHERE { 
?o wdt:P31 wd:Q11424.
?x wdt:P1441 ?o.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?worker)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Kingdom of Heaven'), ('worker', '11')]
[('film', 'iSteve'), ('worker', '11')]
[('film', 'Kazimierz Wielki'), ('worker', '9')]
[('film', 'Tombstone'), ('worker', '8')]
[('film', 'Hrabina Cosel'), ('worker', '7')]
[('film', 'Chopin: Desire for Love'), ('worker', '6')]
[('film', 'Jack Strong'), ('worker', '6')]
[('film', 'A Little Chaos'), ('worker', '6')]
[('film', "Devil's Mistress"), ('worker', '6')]
[('film', 'Copernicus'), ('worker', '5')]


10

In [29]:
queryString = """
SELECT DISTINCT ?film (COUNT(DISTINCT ?x) AS ?worker)
WHERE { 
?o wdt:P31 wd:Q11424.
?x wdt:P767 ?o.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?worker)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Dances with Wolves'), ('worker', '1')]


1

In [31]:
queryString = """
SELECT DISTINCT ?film (COUNT(DISTINCT ?x) AS ?worker)
WHERE { 
?o wdt:P31 wd:Q11424.
?x wdt:P1686 ?o.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?worker)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Dirty God'), ('worker', '1')]


1

All of these are very poor results if I consider that there are a lot of people that generally work on a film. 
Let's see some numerical result from the predicates found above from film to people:
- P161: cast member;
- P3092: film crew member;
- P1029: crew members;
- P3342: significant person.

In [32]:
#P161
queryString = """
SELECT ?film (COUNT(DISTINCT ?x) AS ?cast)
WHERE { 
?o wdt:P31 wd:Q11424;
    wdt:P161 ?x.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?cast)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Baantjer, de film: De Cock en de wraak zonder einde'), ('cast', '267')]
[('film', 'Mamma Mia! Here We Go Again'), ('cast', '224')]
[('film', 'Nevinné lži'), ('cast', '156')]
[('film', 'If Paris Were Told to Us'), ('cast', '152')]
[('film', 'Napoléon'), ('cast', '142')]
[('film', 'Days of Betrayal'), ('cast', '138')]
[('film', 'Lennon or McCartney'), ('cast', '138')]
[('film', 'Royal Affairs in Versailles'), ('cast', '131')]
[('film', 'Ali'), ('cast', '121')]
[('film', 'Love Between the Raindrops'), ('cast', '120')]


10

In [33]:
#P3092
queryString = """
SELECT ?film (COUNT(DISTINCT ?x) AS ?film_crew)
WHERE { 
?o wdt:P31 wd:Q11424;
    wdt:P3092 ?x.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?film_crew)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Captain America: Civil War'), ('film_crew', '43')]
[('film', "Le choix d'un peuple"), ('film_crew', '28')]
[('film', 'Rogue One'), ('film_crew', '26')]
[('film', 'Doctor Strange'), ('film_crew', '23')]
[('film', 'Thor: Ragnarok'), ('film_crew', '21')]
[('film', 'So Dear to My Heart'), ('film_crew', '21')]
[('film', 'Ant-Man'), ('film_crew', '18')]
[('film', 'The Other Side of November'), ('film_crew', '14')]
[('film', 'The Untouchable'), ('film_crew', '14')]
[('film', '15 nov.'), ('film_crew', '13')]


10

In [34]:
#P1029
queryString = """
SELECT ?film (COUNT(DISTINCT ?x) AS ?crew)
WHERE { 
?o wdt:P31 wd:Q11424;
    wdt:P1029 ?x.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?crew)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Pickpocket'), ('crew', '3')]
[('film', 'The Eclipse'), ('crew', '1')]
[('film', 'German Concentration Camps Factual Survey'), ('crew', '1')]
[('film', 'Boncou, Tribal Postman'), ('crew', '1')]
[('film', 'Ant-Man'), ('crew', '1')]


5

In [35]:
#P1029
queryString = """
SELECT ?film (COUNT(DISTINCT ?x) AS ?significant)
WHERE { 
?o wdt:P31 wd:Q11424;
    wdt:P3342 ?x.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?significant)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Bullets, Fangs and Dinner at 8'), ('significant', '2')]
[('film', 'The Samuel Project'), ('significant', '2')]
[('film', 'Graduation Afternoon'), ('significant', '1')]
[('film', 'Va savoir'), ('significant', '1')]
[('film', 'Five Feet Apart'), ('significant', '1')]
[('film', 'Mike & Fred vs the Dead'), ('significant', '1')]
[('film', 'Clouds'), ('significant', '1')]
[('film', 'Susi Susanti: Love All'), ('significant', '1')]
[('film', 'Sophie Scholl – The Final Days'), ('significant', '1')]
[('film', 'South of 8'), ('significant', '1')]


10

In [36]:
#all people connected with a film
queryString = """
SELECT ?film (COUNT(DISTINCT ?x) AS ?all)
WHERE { 
?o wdt:P31 wd:Q11424;
    ?pr ?x.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?all)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Baantjer, de film: De Cock en de wraak zonder einde'), ('all', '272')]
[('film', 'Mamma Mia! Here We Go Again'), ('all', '230')]
[('film', 'Nevinné lži'), ('all', '191')]
[('film', 'Royal Affairs in Versailles'), ('all', '190')]
[('film', 'If Paris Were Told to Us'), ('all', '154')]
[('film', 'Captain America: Civil War'), ('all', '153')]
[('film', 'Napoléon'), ('all', '149')]
[('film', 'Days of Betrayal'), ('all', '143')]
[('film', 'Rogue One'), ('all', '140')]
[('film', 'Lennon or McCartney'), ('all', '138')]


10

In [37]:
#all people connected with a film compare to cast members
queryString = """
SELECT ?film ?all ?cast
WHERE { 
    {
        SELECT DISTINCT ?o (COUNT(DISTINCT ?x) AS ?all)
        WHERE { 
        ?o wdt:P31 wd:Q11424;
            ?pr ?x.
        ?x wdt:P31 wd:Q5.
        } 
        GROUP BY ?o
    }
    {
        SELECT DISTINCT ?o (COUNT(DISTINCT ?x) AS ?cast)
        WHERE { 
        ?o wdt:P31 wd:Q11424;
            wdt:P161 ?x.
        ?x wdt:P31 wd:Q5.
        } 
        GROUP BY ?o
    }
    ?o <http://schema.org/name> ?film .
} 
GROUP BY ?o ?film
ORDER BY DESC (?all)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Baantjer, de film: De Cock en de wraak zonder einde'), ('all', '272'), ('cast', '267')]
[('film', 'Mamma Mia! Here We Go Again'), ('all', '230'), ('cast', '224')]
[('film', 'Nevinné lži'), ('all', '191'), ('cast', '156')]
[('film', 'Royal Affairs in Versailles'), ('all', '190'), ('cast', '131')]
[('film', 'If Paris Were Told to Us'), ('all', '154'), ('cast', '152')]
[('film', 'Captain America: Civil War'), ('all', '153'), ('cast', '97')]
[('film', 'Napoléon'), ('all', '149'), ('cast', '142')]
[('film', 'Days of Betrayal'), ('all', '143'), ('cast', '138')]
[('film', 'Rogue One'), ('all', '140'), ('cast', '64')]
[('film', 'Lennon or McCartney'), ('all', '138'), ('cast', '138')]


10

#### Consideration
In the most cases the cast members represents the majority of the people that work on a film. The other people are surely choreographer, costum designer, editors, producers. Since I don't find any predicate that tell me the simple fact that a people work on a film I will use this BGP to represent people work on a film.

```
?o wdt:P31 wd:Q11424;
    ?pr ?x.
?x wdt:P31 wd:Q5.
```

## Task 4
Compare the workers amongst the films directed by the two directors

Starting from the BGP above, the predicate director (wdt:P57) and the two director nodes wd:Q25089 (Woody Allen) and wd:Q3772 (Quentin Tarantino) I will perform some query.

#### Query 1. 

**Which are the top 10 films with more people worked on?**


**Is Quentin Tarantino the director of the film with more people worked on?**

In [48]:
# TOP-10 films with director and people
queryString = """
SELECT ?film ?director (COUNT(DISTINCT ?x) AS ?all)
WHERE { 
?o wdt:P31 wd:Q11424;
    wdt:P57 ?dir;
    ?pr ?x.
?x wdt:P31 wd:Q5.
?o <http://schema.org/name> ?film .
?dir <http://schema.org/name> ?director .
FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772)
} 
GROUP BY ?o ?film ?director ?dir
ORDER BY DESC (?all)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Once Upon a Time in Hollywood'), ('director', 'Quentin Tarantino'), ('all', '81')]
[('film', 'Celebrity'), ('director', 'Woody Allen'), ('all', '74')]
[('film', 'Django Unchained'), ('director', 'Quentin Tarantino'), ('all', '62')]
[('film', 'Inglourious Basterds'), ('director', 'Quentin Tarantino'), ('all', '60')]
[('film', 'To Rome with Love'), ('director', 'Woody Allen'), ('all', '48')]
[('film', 'New York Stories'), ('director', 'Woody Allen'), ('all', '46')]
[('film', 'Zelig'), ('director', 'Woody Allen'), ('all', '45')]
[('film', 'Pulp Fiction'), ('director', 'Quentin Tarantino'), ('all', '43')]
[('film', 'The Concert for New York City'), ('director', 'Woody Allen'), ('all', '43')]
[('film', 'Deconstructing Harry'), ('director', 'Woody Allen'), ('all', '41')]


10

In [49]:
#Ask if Tarantino is the director of the film with more people worked on
queryString = """
ASK WHERE { 
{
    SELECT ?f ?dir (COUNT(DISTINCT ?x) AS ?all)
    WHERE { 
    ?f wdt:P31 wd:Q11424;
        wdt:P57 ?dir;
        ?pr ?x.
    ?x wdt:P31 wd:Q5.
    FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772)
    } 
    GROUP BY ?f ?dir
    ORDER BY DESC (?all)
    LIMIT 1
}
FILTER(?dir = wd:Q3772)
    
} 
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

#### Query 2. 

**Which is the film with the biggest cast? Who is the director?**

In [50]:
queryString = """
SELECT ?film ?director ?cast
WHERE {
    {
        SELECT ?f ?dir (COUNT(DISTINCT ?x) AS ?cast)
        WHERE {
        ?f wdt:P31 wd:Q11424;
            wdt:P57 ?dir;
            wdt:P161 ?x.
        ?x wdt:P31 wd:Q5.
        FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772)
        } 
        GROUP BY ?f ?dir
    }
    {
        SELECT MAX(?c) AS ?max WHERE
        {
            SELECT ?f ?dir (COUNT(DISTINCT ?x) AS ?c)
            WHERE {
            ?f wdt:P31 wd:Q11424;
                wdt:P57 ?dir;
                wdt:P161 ?x.
            ?x wdt:P31 wd:Q5.
            FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772)
            } 
            GROUP BY ?f ?dir
        }
    }
    ?f <http://schema.org/name> ?film .
    ?dir <http://schema.org/name> ?director .
    FILTER (?cast = ?max)
} 
"""

print("Results")
run_query(queryString)

Results
[('film', 'Celebrity'), ('director', 'Woody Allen'), ('cast', '64')]


1

#### Query 3. 

**How many films they directed with a cast of more than 10 people?**

In [60]:
queryString = """
SELECT ?film ?director (COUNT(DISTINCT ?x) AS ?cast)
WHERE {
?f wdt:P31 wd:Q11424;
    wdt:P57 ?dir;
    wdt:P161 ?x.
?x wdt:P31 wd:Q5.
?f <http://schema.org/name> ?film .
?dir <http://schema.org/name> ?director .
FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772)
} 
GROUP BY ?f ?dir ?film ?director
HAVING (COUNT(DISTINCT ?x) > 10)
ORDER BY DESC(?cast)
"""

print("Results")
run_query(queryString)

Results
[('film', 'Celebrity'), ('director', 'Woody Allen'), ('cast', '64')]
[('film', 'Inglourious Basterds'), ('director', 'Quentin Tarantino'), ('cast', '52')]
[('film', 'Django Unchained'), ('director', 'Quentin Tarantino'), ('cast', '52')]
[('film', 'Once Upon a Time in Hollywood'), ('director', 'Quentin Tarantino'), ('cast', '51')]
[('film', 'To Rome with Love'), ('director', 'Woody Allen'), ('cast', '45')]
[('film', 'Zelig'), ('director', 'Woody Allen'), ('cast', '39')]
[('film', 'New York Stories'), ('director', 'Woody Allen'), ('cast', '39')]
[('film', 'The Concert for New York City'), ('director', 'Woody Allen'), ('cast', '39')]
[('film', 'Love and Death'), ('director', 'Woody Allen'), ('cast', '35')]
[('film', 'Shadows and Fog'), ('director', 'Woody Allen'), ('cast', '35')]
[('film', 'Sin City'), ('director', 'Quentin Tarantino'), ('cast', '35')]
[('film', 'Deconstructing Harry'), ('director', 'Woody Allen'), ('cast', '35')]
[('film', 'Pulp Fiction'), ('director', 'Quentin T

54

In [61]:
queryString = """
SELECT ?director (COUNT(*) AS ?films)
WHERE {
    {
        SELECT ?f ?dir (COUNT(DISTINCT ?x) AS ?cast)
        WHERE {
        ?f wdt:P31 wd:Q11424;
            wdt:P57 ?dir;
            wdt:P161 ?x.
        ?x wdt:P31 wd:Q5.
        FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772)
        } 
        GROUP BY ?f ?dir
        HAVING (COUNT(DISTINCT ?x) > 10)
    }
    ?dir <http://schema.org/name> ?director .
} 
GROUP BY ?dir ?director
"""

print("Results")
run_query(queryString)

Results
[('director', 'Quentin Tarantino'), ('films', '11')]
[('director', 'Woody Allen'), ('films', '43')]


2

## Task 5
Return some numerical comparison between the two directors (e.g., how many workers in Tarantino's movies who also worked in Allen's films?, what is the film with the highest number of shared actors? Who is the most used actor by both the directors? etc. )

In [74]:
#how many workers in Tarantino's movies who also worked in Allen's films?
queryString = """
SELECT (COUNT(*) AS ?common_workers)
WHERE {
    {
        #SELECT WORKERS FROM TARANTINO'S MOVIE
        
        SELECT DISTINCT ?x AS ?tarantino
        WHERE {
        ?f wdt:P31 wd:Q11424;
            wdt:P57 wd:Q3772;
            ?pr ?x.
        ?x wdt:P31 wd:Q5.
        } 
    }
    
    {
        #SELECT WORKERS FROM ALLEN'S MOVIE
        
        SELECT DISTINCT ?x AS ?allen
        WHERE {
        ?f wdt:P31 wd:Q11424;
            wdt:P57 wd:Q25089;
            ?pr ?x.
        ?x wdt:P31 wd:Q5.
        } 
    }
    FILTER(?tarantino = ?allen)
}
"""

print("Results")
run_query(queryString)

Results
[('common_workers', '16')]


1

What is the film with the highest number of shared actors? To answer tho this question I firstly perform a query to find the actors who worked for both director

In [81]:
#what is the film with the highest number of shared actors?
queryString = """
SELECT ?common ?actor
WHERE {
    {
        #SELECT WORKERS FROM TARANTINO'S MOVIE
        
        SELECT DISTINCT ?x AS ?tarantino
        WHERE {
        ?f wdt:P31 wd:Q11424;
            wdt:P57 wd:Q3772;
            wdt:P161 ?x.
        ?x wdt:P31 wd:Q5.
        } 
    }
    
    {
        #SELECT WORKERS FROM ALLEN'S MOVIE
        
        SELECT DISTINCT ?x AS ?allen
        WHERE {
        ?f wdt:P31 wd:Q11424;
            wdt:P57 wd:Q25089;
            wdt:P161 ?x.
        ?x wdt:P31 wd:Q5.
        } 
    }
    FILTER(?tarantino = ?allen)
    BIND(?tarantino AS ?common)
    ?common sc:name ?actor 
}
"""

print("Results")
run_query(queryString)

Results
[('common', 'http://www.wikidata.org/entity/Q185724'), ('actor', 'Mike Myers')]
[('common', 'http://www.wikidata.org/entity/Q203804'), ('actor', 'Tim Roth')]
[('common', 'http://www.wikidata.org/entity/Q104061'), ('actor', 'Steve Buscemi')]
[('common', 'http://www.wikidata.org/entity/Q231096'), ('actor', 'Léa Seydoux')]
[('common', 'http://www.wikidata.org/entity/Q109232'), ('actor', 'Rosanna Arquette')]
[('common', 'http://www.wikidata.org/entity/Q207596'), ('actor', 'Daryl Hannah')]
[('common', 'http://www.wikidata.org/entity/Q36949'), ('actor', 'Robert De Niro')]
[('common', 'http://www.wikidata.org/entity/Q125017'), ('actor', 'Uma Thurman')]
[('common', 'http://www.wikidata.org/entity/Q185051'), ('actor', 'Christopher Walken')]
[('common', 'http://www.wikidata.org/entity/Q38111'), ('actor', 'Leonardo DiCaprio')]
[('common', 'http://www.wikidata.org/entity/Q202148'), ('actor', 'Burt Reynolds')]
[('common', 'http://www.wikidata.org/entity/Q76819'), ('actor', 'Christoph Waltz'

12

In [85]:
#find films in which these actors worked
queryString = """
SELECT ?film (COUNT(DISTINCT ?actor) AS ?more)
WHERE {
    {
        SELECT ?common
        WHERE {
            {
                #SELECT WORKERS FROM TARANTINO'S MOVIE

                SELECT DISTINCT ?x AS ?tarantino
                WHERE {
                ?f wdt:P31 wd:Q11424;
                    wdt:P57 wd:Q3772;
                    wdt:P161 ?x.
                ?x wdt:P31 wd:Q5.
                } 
            }

            {
                #SELECT WORKERS FROM ALLEN'S MOVIE

                SELECT DISTINCT ?x AS ?allen
                WHERE {
                ?f wdt:P31 wd:Q11424;
                    wdt:P57 wd:Q25089;
                    wdt:P161 ?x.
                ?x wdt:P31 wd:Q5.
                } 
            }
            FILTER(?tarantino = ?allen)
            BIND(?tarantino AS ?common)
        }
    }
    ?f wdt:P31 wd:Q11424;
        wdt:P161 ?x.
    ?f sc:name ?film.
    ?common sc:name ?actor.
    FILTER(?x = ?common)
}

GROUP BY ?f ?film
ORDER BY DESC (?more)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Pulp Fiction'), ('more', '5')]
[('film', 'The Concert for New York City'), ('more', '4')]
[('film', 'One Hundred and One Nights'), ('more', '3')]
[('film', 'Inglourious Basterds'), ('more', '3')]
[('film', 'Once Upon a Time in Hollywood'), ('more', '3')]
[('film', 'The War with Grandpa'), ('more', '3')]
[('film', 'Search and Destroy'), ('more', '2')]
[('film', 'Kiss Daddy Goodnight'), ('more', '2')]
[('film', 'King of New York'), ('more', '2')]
[('film', 'Deceiver'), ('more', '2')]


10

In [18]:
#find films in which these actors worked
queryString = """
SELECT ?film ?director (COUNT(DISTINCT ?actor) AS ?more) (GROUP_CONCAT(?actor;separator=",") AS ?actors)
WHERE {
    {
        SELECT ?common
        WHERE {
            {
                #SELECT WORKERS FROM TARANTINO'S MOVIE

                SELECT DISTINCT ?x AS ?tarantino
                WHERE {
                ?f wdt:P31 wd:Q11424;
                    wdt:P57 wd:Q3772;
                    wdt:P161 ?x.
                ?x wdt:P31 wd:Q5.
                } 
            }
            {
                #SELECT WORKERS FROM ALLEN'S MOVIE

                SELECT DISTINCT ?x AS ?allen
                WHERE {
                ?f wdt:P31 wd:Q11424;
                    wdt:P57 wd:Q25089;
                    wdt:P161 ?x.
                ?x wdt:P31 wd:Q5.
                } 
            }
            FILTER(?tarantino = ?allen)
            BIND(?tarantino AS ?common)
        }
    }
    ?f wdt:P31 wd:Q11424;
        wdt:P57 ?dir;
        wdt:P161 ?x.
    ?f sc:name ?film.
    ?common sc:name ?actor.
    ?dir sc:name ?director.
    #FILTER((?dir = wd:Q25089 || ?dir = wd:Q3772) && (?x = ?common))
    FILTER(?x = ?common)
}

GROUP BY ?f ?film ?dir ?director
ORDER BY DESC (?more)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('film', 'Pulp Fiction'), ('director', 'Quentin Tarantino'), ('more', '5'), ('actors', 'Tim Roth,Steve Buscemi,Rosanna Arquette,Uma Thurman,Christopher Walken')]


1

Who is the most used actor by both the directors?

In [19]:
#find films in which these actors worked
queryString = """
SELECT ?director ?actor (COUNT(DISTINCT ?f) AS ?films)
WHERE {
    {
        SELECT ?common
        WHERE {
            {
                #SELECT WORKERS FROM TARANTINO'S MOVIE

                SELECT DISTINCT ?x AS ?tarantino
                WHERE {
                ?f wdt:P31 wd:Q11424;
                    wdt:P57 wd:Q3772;
                    wdt:P161 ?x.
                ?x wdt:P31 wd:Q5.
                } 
            }
            {
                #SELECT WORKERS FROM ALLEN'S MOVIE

                SELECT DISTINCT ?x AS ?allen
                WHERE {
                ?f wdt:P31 wd:Q11424;
                    wdt:P57 wd:Q25089;
                    wdt:P161 ?x.
                ?x wdt:P31 wd:Q5.
                } 
            }
            FILTER(?tarantino = ?allen)
            BIND(?tarantino AS ?common)
        }
    }
    ?f wdt:P31 wd:Q11424;
        wdt:P57 ?dir;
        wdt:P161 ?x.
    ?f sc:name ?film.
    ?common sc:name ?actor.
    ?dir sc:name ?director.
    FILTER((?dir = wd:Q25089 || ?dir = wd:Q3772) && (?x = ?common))
}

GROUP BY ?common ?actor ?dir ?director
ORDER BY DESC (?films)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('director', 'Quentin Tarantino'), ('actor', 'Uma Thurman'), ('films', '4')]
[('director', 'Quentin Tarantino'), ('actor', 'Tim Roth'), ('films', '4')]
[('director', 'Quentin Tarantino'), ('actor', 'Daryl Hannah'), ('films', '3')]
[('director', 'Quentin Tarantino'), ('actor', 'Steve Buscemi'), ('films', '2')]
[('director', 'Woody Allen'), ('actor', 'Steve Buscemi'), ('films', '2')]
[('director', 'Woody Allen'), ('actor', 'Leonardo DiCaprio'), ('films', '2')]
[('director', 'Quentin Tarantino'), ('actor', 'Leonardo DiCaprio'), ('films', '2')]
[('director', 'Quentin Tarantino'), ('actor', 'Christoph Waltz'), ('films', '2')]
[('director', 'Woody Allen'), ('actor', 'Tim Roth'), ('films', '1')]
[('director', 'Woody Allen'), ('actor', 'Uma Thurman'), ('films', '1')]


10

The most used actors for Tarantino are  Thurman and Roth in 4 films, while Allen used Buscemi and DiCaprio in 2 films.
Now I will look on the sum of both directors.

In [20]:
#find films in which these actors worked
queryString = """
SELECT ?actor (COUNT(DISTINCT ?f) AS ?films)
WHERE {
    {
        SELECT ?common
        WHERE {
            {
                #SELECT WORKERS FROM TARANTINO'S MOVIE

                SELECT DISTINCT ?x AS ?tarantino
                WHERE {
                ?f wdt:P31 wd:Q11424;
                    wdt:P57 wd:Q3772;
                    wdt:P161 ?x.
                ?x wdt:P31 wd:Q5.
                } 
            }
            {
                #SELECT WORKERS FROM ALLEN'S MOVIE

                SELECT DISTINCT ?x AS ?allen
                WHERE {
                ?f wdt:P31 wd:Q11424;
                    wdt:P57 wd:Q25089;
                    wdt:P161 ?x.
                ?x wdt:P31 wd:Q5.
                } 
            }
            FILTER(?tarantino = ?allen)
            BIND(?tarantino AS ?common)
        }
    }
    ?f wdt:P31 wd:Q11424;
        wdt:P57 ?dir;
        wdt:P161 ?x.
    ?f sc:name ?film.
    ?common sc:name ?actor.
    ?dir sc:name ?director.
    FILTER((?dir = wd:Q25089 || ?dir = wd:Q3772) && (?x = ?common))
}

GROUP BY ?common ?actor
ORDER BY DESC (?films)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('actor', 'Tim Roth'), ('films', '5')]
[('actor', 'Uma Thurman'), ('films', '5')]
[('actor', 'Steve Buscemi'), ('films', '4')]
[('actor', 'Daryl Hannah'), ('films', '4')]
[('actor', 'Leonardo DiCaprio'), ('films', '4')]
[('actor', 'Christoph Waltz'), ('films', '3')]
[('actor', 'Burt Reynolds'), ('films', '2')]
[('actor', 'Christopher Walken'), ('films', '2')]
[('actor', 'Léa Seydoux'), ('films', '2')]
[('actor', 'Robert De Niro'), ('films', '2')]


10

The two most used actors are Roth and Thurman

## Task 6
Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?

In [64]:
# look for the budget
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?o wdt:P31 wd:Q11424;
    ?pr ?x.
?pr <http://schema.org/name> ?name .
FILTER(isNumeric(?x))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('pr', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('pr', 'http://www.wikidata.org/prop/direct/P2130'), ('name', 'cost')]
[('pr', 'http://www.wikidata.org/prop/direct/P2142'), ('name', 'box office')]
[('pr', 'http://www.wikidata.org/prop/direct/P8687'), ('name', 'social media followers')]
[('pr', 'http://www.wikidata.org/prop/direct/P1104'), ('name', 'number of pages')]
[('pr', 'http://www.wikidata.org/prop/direct/P2769'), ('name', 'budget')]
[('pr', 'http://www.wikidata.org/prop/direct/P1110'), ('name', 'attendance')]
[('pr', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
[('pr', 'http://www.wikidata.org/prop/direct/P5436'), ('name', 'number of viewers/listeners')]
[('pr', 'http://www.wikidata.org/prop/direct/P2043'), ('name', 'length')]
[('pr', 'http://www.wikidata.org/prop/direct/P2635'), ('name', 'number of parts of this work')]
[('pr

25

Found `wdt:P2769` as budget

In [68]:
# found the max budget
queryString = """
SELECT ?director (COUNT(?x) AS ?max_budget)
WHERE { 
?o wdt:P31 wd:Q11424;
    wdt:P57 ?dir;
    wdt:P2769 ?x.
?o <http://schema.org/name> ?film .
?dir <http://schema.org/name> ?director .
FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772)
} 
GROUP BY ?director ?dir
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('director', 'Woody Allen'), ('max_budget', '1')]


1

I have the budget only for one film, so I cannot answer to this question. Anyway I will write how the query will looks like.

In [2]:
# found the max budget
queryString = """
ASK WHERE { 
{
    SELECT (MAX(?x) AS ?max_tarantino)
    WHERE { 
    ?o wdt:P31 wd:Q11424;
        wdt:P57 wd:Q3772;
        wdt:P2769 ?x.
    } 
}
{
    SELECT (MAX(?x) AS ?max_allen)
    WHERE { 
    ?o wdt:P31 wd:Q11424;
        wdt:P57 wd:Q25089;
        wdt:P2769 ?x.
    } 
}
FILTER(?max_tarantino > ?max_allen)
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': False}

## Task 7 
Who has films with more nominations for Academy Awards and who won more Academy Awards (with his films not only personal awards).

### Task 7.1
Find the BGP for Academy Awards

Start from a film looking for property that are similar to `nominated`

In [67]:
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?f wdt:P31 wd:Q11424;
    ?pr ?a.
?pr sc:name ?name .
FILTER(REGEX(?name,".*nominated.*"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P4353'), ('name', 'nominated by')]


2

In [68]:
#looking for what are the instances of these objects
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
?f wdt:P31 wd:Q11424;
    wdt:P1411 ?a.
?a wdt:P31 ?x.
?x sc:name ?name .
FILTER(REGEX(?name,".*award.*"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q96474676'), ('name', 'award for best leading actress')]
[('x', 'http://www.wikidata.org/entity/Q96474688'), ('name', 'award for best film by genre')]
[('x', 'http://www.wikidata.org/entity/Q96474699'), ('name', 'award for best costume design')]
[('x', 'http://www.wikidata.org/entity/Q96474679'), ('name', 'award for best direction')]
[('x', 'http://www.wikidata.org/entity/Q618779'), ('name', 'award')]
[('x', 'http://www.wikidata.org/entity/Q96474603'), ('name', 'award for best supporting actor')]
[('x', 'http://www.wikidata.org/entity/Q56116950'), ('name', 'film award category')]
[('x', 'http://www.wikidata.org/entity/Q96474683'), ('name', 'award for best sound editing')]
[('x', 'http://www.wikidata.org/entity/Q96474709'), ('name', 'award for best visual effects')]
[('x', 'http://www.wikidata.org/entity/Q4220917'), ('name', 'film award')]
[('x', 'http://www.wikidata.org/entity/Q96474687'), ('name', 'award for best film')]
[('x', 'http://ww

48

`wd:Q4220917`(film award) seems to be the right class of award. Try to look for subclasses of this class to find the Academy award

In [69]:
queryString = """
SELECT DISTINCT ?a ?name
WHERE { 
?a wdt:P279+ wd:Q4220917.
?a sc:name ?name .
FILTER(REGEX(?name,".*cadem.*"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q655089'), ('name', 'International Indian Film Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q2510955'), ('name', 'Japan Academy Prize for Animation of the Year')]
[('a', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q194258'), ('name', 'Japan Academy Film Prize')]
[('a', 'http://www.wikidata.org/entity/Q732997'), ('name', 'British Academy Film Awards')]
[('a', 'http://www.wikidata.org/entity/Q384139'), ('name', 'Africa Movie Academy Award')]
[('a', 'http://www.wikidata.org/entity/Q11024975'), ('name', 'Myanmar Motion Picture Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q18345192'), ('name', 'Golden Icons Academy Movie Awards')]
[('a', 'http://www.wikidata.org/entity/Q74066919'), ('name', '14th Africa Movie Academy Awards')]


9

### Task 7.2 
Find the related subproperties

In [70]:
#look for something istance of academy awards
queryString = """
SELECT DISTINCT ?a ?name
WHERE { 
?a wdt:P31 wd:Q19020.
?a sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q22235305'), ('name', 'Academy Award for Best Original Musical or Comedy Score')]
[('a', 'http://www.wikidata.org/entity/Q21995136'), ('name', 'Academy Award for Best Cinematography, Color')]
[('a', 'http://www.wikidata.org/entity/Q21995139'), ('name', 'Academy Award for Best Cinematography, Black-and-White')]
[('a', 'http://www.wikidata.org/entity/Q22253131'), ('name', 'Academy Award for Best Art Direction, Color')]
[('a', 'http://www.wikidata.org/entity/Q22752811'), ('name', 'Academy Award for Best Score, Adaptation or Treatment')]
[('a', 'http://www.wikidata.org/entity/Q8610'), ('name', 'Academy Scientific and Technical Award')]
[('a', 'http://www.wikidata.org/entity/Q1324407'), ('name', 'Academy Award for Best Live Action Short Film')]
[('a', 'http://www.wikidata.org/entity/Q487136'), ('name', 'Academy Award for Best Makeup and Hairstyling')]
[('a', 'http://www.wikidata.org/entity/Q504298'), ('name', 'Academy Award for Best Story')]
[(

63

`wd:Q19020` is the class of Academy Awards. Then selecting objects that are istance of academy awards we have the list of all type of academy awards. 

In [71]:
#look for some properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a wdt:P31 wd:Q19020.
?a ?pr ?o.
?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('pr', 'http://www.wikidata.org/prop/direct/P2257'), ('name', 'event interval')]
[('pr', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('pr

32

In [72]:
#look for some properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a wdt:P31 wd:Q19020.
?o ?pr ?a.
?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory')]
[('pr', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('pr', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('pr', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('pr', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
[('pr', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]


12

The properties of interest are:
- `wdt:P1411`  nominated for
- `wdt:P166`  award received

### Task 7.3
Find how they are related to the directors (`wd:Q3455803`)

In [73]:
#find all people who is connected with the class director
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?sub wdt:P279* wd:Q3455803.
wd:Q3772 ?pr ?sub.

?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]


1

In [74]:
#from the directors find the relations to the academy award
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?award wdt:P31 wd:Q19020.
?x wdt:P106 wd:Q3455803.

?x ?pr ?award.

?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]


2

Now I have all the informations to find some result about academy awards won by directors or films that they directed.
Remember that:
- `wd:Q11424` film
- `wd:Q19020` academy award
- `wd:Q3455803` class of director
- `wdt:P57` property director

In [75]:
#academy award won by directors
queryString = """
SELECT DISTINCT ?director (COUNT( ?a) AS ?cont) (GROUP_CONCAT(?award;separator="-") AS ?awards)
WHERE { 
?a wdt:P31 wd:Q19020.
?x wdt:P106 wd:Q3455803.

?x wdt:P166 ?a.


?x sc:name ?director .
?a sc:name ?award .
} 
GROUP BY ?director
ORDER BY DESC (?cont)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('director', 'Walt Disney'), ('cont', '7'), ('awards', 'Academy Award for Best Documentary (Short Subject)-Academy Award for Best Animated Short Film-Academy Award for Best Live Action Short Film-Irving G. Thalberg Memorial Award-Academy Honorary Award-Academy Award for Best Documentary Feature-Academy Award for Best Short Subject, Two-reel')]
[('director', 'Steven Spielberg'), ('cont', '3'), ('awards', 'Academy Award for Best Director-Academy Award for Best Picture-Irving G. Thalberg Memorial Award')]
[('director', 'James Cameron'), ('cont', '3'), ('awards', 'Academy Award for Best Director-Academy Award for Best Picture-Academy Award for Best Film Editing')]
[('director', 'Charlie Chaplin'), ('cont', '2'), ('awards', 'Academy Honorary Award-Academy Award for Best Original Dramatic Score')]
[('director', 'Hal Roach'), ('cont', '2'), ('awards', 'Academy Award for Best Live Action Short Film, Comedy-Academy Award for Best Live Action Short Film, One-Reel')]
[('director', 'Denni

20

In [76]:
#academy award nominations by directors
queryString = """
SELECT DISTINCT ?x ?director (COUNT(*) AS ?cont)
WHERE { 
?award wdt:P31 wd:Q19020.
?x wdt:P106 wd:Q3455803.

?x wdt:P1411 ?award.

?x sc:name ?director .
} 
GROUP BY ?x ?director
ORDER BY DESC (?cont)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q8704'), ('director', 'Walt Disney'), ('cont', '6')]
[('x', 'http://www.wikidata.org/entity/Q2001'), ('director', 'Stanley Kubrick'), ('cont', '5')]
[('x', 'http://www.wikidata.org/entity/Q72717'), ('director', 'Elia Kazan'), ('cont', '3')]
[('x', 'http://www.wikidata.org/entity/Q41148'), ('director', 'Martin Scorsese'), ('cont', '3')]
[('x', 'http://www.wikidata.org/entity/Q314502'), ('director', 'Jason Reitman'), ('cont', '3')]
[('x', 'http://www.wikidata.org/entity/Q372394'), ('director', 'Martin McDonagh'), ('cont', '3')]
[('x', 'http://www.wikidata.org/entity/Q55245'), ('director', 'Laurence Olivier'), ('cont', '3')]
[('x', 'http://www.wikidata.org/entity/Q7371'), ('director', 'Federico Fellini'), ('cont', '3')]
[('x', 'http://www.wikidata.org/entity/Q469164'), ('director', 'Cesare Zavattini'), ('cont', '3')]
[('x', 'http://www.wikidata.org/entity/Q42574'), ('director', 'James Cameron'), ('cont', '3')]
[('x', 'http://www.wikidata.org/

20

In [77]:
#academy award nominations by directors
queryString = """
SELECT DISTINCT ?director ?nominations ?wins
WHERE { 
{
    SELECT DISTINCT ?x1 (COUNT(*) AS ?nominations)
    WHERE { 
    ?award wdt:P31 wd:Q19020.
    ?x1 wdt:P106 wd:Q3455803.

    ?x1 wdt:P1411 ?award.
    } 
    GROUP BY ?x1 
}
{
    SELECT DISTINCT ?x2 (COUNT(*) AS ?wins)
    WHERE { 
    ?a wdt:P31 wd:Q19020.
    ?x2 wdt:P106 wd:Q3455803.

    ?x2 wdt:P166 ?a.

    } 
    GROUP BY ?x2
}
FILTER(?x1=?x2)
?x1 sc:name ?director
} 
ORDER BY DESC ( ?nominations) DESC (?wins)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('director', 'Walt Disney'), ('nominations', '6'), ('wins', '7')]
[('director', 'Stanley Kubrick'), ('nominations', '5'), ('wins', '1')]
[('director', 'James Cameron'), ('nominations', '3'), ('wins', '3')]
[('director', 'Charlie Chaplin'), ('nominations', '3'), ('wins', '2')]
[('director', 'Orson Welles'), ('nominations', '3'), ('wins', '2')]
[('director', 'Laurence Olivier'), ('nominations', '3'), ('wins', '2')]
[('director', 'Elia Kazan'), ('nominations', '3'), ('wins', '1')]
[('director', 'Martin McDonagh'), ('nominations', '3'), ('wins', '1')]
[('director', 'Ingmar Bergman'), ('nominations', '3'), ('wins', '1')]
[('director', 'Federico Fellini'), ('nominations', '3'), ('wins', '1')]
[('director', 'Martin Scorsese'), ('nominations', '3'), ('wins', '1')]
[('director', 'Steven Spielberg'), ('nominations', '2'), ('wins', '3')]
[('director', 'Hal Roach'), ('nominations', '2'), ('wins', '2')]
[('director', 'Quentin Tarantino'), ('nominations', '2'), ('wins', '1')]
[('director', 

20

### Task 7.4
Are there alternative queries to get the same result?

Try to organize the work:
- start from films and look how they are related to academy awards
- look if exists some relation that tell me the type of academy award (e.g. best director, best coreography..)
- look if I can connect people who works in a film with the award received in that film

In [2]:
#find how films are connected with academy awards
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?f wdt:P31 wd:Q11424.
?a wdt:P31 wd:Q19020.
?f ?pr ?a.

?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]


2

In [5]:
#find how films are connected with academy awards
queryString = """
SELECT DISTINCT ?film (COUNT(DISTINCT ?a) AS ?cont) (GROUP_CONCAT(?award;separator=" - ") AS ?awards)
WHERE { 
?f wdt:P31 wd:Q11424.
?a wdt:P31 wd:Q19020.
?f wdt:P166 ?a.

?f sc:name ?film .
?a sc:name ?award .
} 
GROUP BY ?f ?film
ORDER BY DESC (?cont)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('film', 'Titanic'), ('cont', '11'), ('awards', 'Academy Award for Best Picture - Academy Award for Best Director - Academy Award for Best Original Song - Academy Award for Best Film Editing - Academy Award for Best Cinematography - Academy Award for Best Costume Design - Academy Award for Best Production Design - Academy Award for Best Visual Effects - Academy Award for Best Sound Editing - Academy Award for Best Sound - Academy Award for Best Original Dramatic Score')]
[('film', 'Ben-Hur'), ('cont', '11'), ('awards', 'Academy Award for Best Cinematography, Color - Academy Award for Best Art Direction, Color - Academy Award for Best Picture - Academy Award for Best Director - Academy Award for Best Film Editing - Academy Award for Best Actor - Academy Award for Best Supporting Actor - Academy Award for Best Original Dramatic or Comedy Score - Academy Award for Best Sound - Academy Award for Best Costume Design, Color - Academy Award for Best Special Effects')]
[('film', 'The 

10

In [8]:
#find how films are connected with academy awards
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?f wdt:P31 wd:Q11424.
?f ?pr ?a.

?pr sc:name ?name .
FILTER(REGEX(?name,".*nom.*") || REGEX(?name,".*win.*") || REGEX(?name,".*cademy.*"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P6145'), ('name', 'Academy Awards Database film ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P5829'), ('name', 'Television Academy Foundation show ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P6150'), ('name', 'Academy Awards Database nominee ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P4353'), ('name', 'nominated by')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]


6

Find something very interesting that is the Academy Awards Database nominee ID (`wdt:P6150`). If we can relate this ID with the people we could know who was nominated and for which film, and possibly for which academy award. Let's find if the object of the predicate wdt:P6150 is just a literal or a node in the graph.

In [34]:
#find how films are connected with academy awards
queryString = """
SELECT DISTINCT ?a ?name
WHERE { 
?f wdt:P31 wd:Q11424.
?f wdt:P6150 ?a.

OPTIONAL{?a sc:name ?name .}
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('a', '6780')]
[('a', '5625')]
[('a', '7620')]
[('a', '4403')]
[('a', '4916')]
[('a', '8475')]


6

Only 6 results, so this is not useful. Clear the mind and start again from academy awards

In [33]:
#properties of academy awards
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a wdt:P31 wd:Q19020.
?a ?pr ?o.


?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('pr', 'http://www.wikidata.org/prop/direct/P2257'), ('name', 'event interval')]
[('pr', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('pr

32

In [46]:
#see the fields of work
queryString = """
SELECT DISTINCT ?o ?name
WHERE { 
?a wdt:P31 wd:Q19020.
?a wdt:P2517 ?o.


?o sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q6372982'), ('name', 'Category:Best Documentary Short Subject Academy Award winners')]
[('o', 'http://www.wikidata.org/entity/Q9240896'), ('name', 'Category:Best Animated Feature Academy Award winners')]
[('o', 'http://www.wikidata.org/entity/Q9650583'), ('name', 'Category:Best Original Song Academy Award winners')]
[('o', 'http://www.wikidata.org/entity/Q7146100'), ('name', 'Category:Best Directing Academy Award winners')]
[('o', 'http://www.wikidata.org/entity/Q8298336'), ('name', 'Category:Best Assistant Director Academy Award winners')]
[('o', 'http://www.wikidata.org/entity/Q8298430'), ('name', 'Category:Best Sound Mixing Academy Award winners')]
[('o', 'http://www.wikidata.org/entity/Q8794824'), ('name', 'Category:Producers who won the Best Picture Academy Award')]
[('o', 'http://www.wikidata.org/entity/Q6373064'), ('name', 'Category:Live Action Short Film Academy Award winners')]
[('o', 'http://www.wikidata.org/entity/Q8219410'), ('

29

In [53]:
#see some properties of the category Best Directing Academy Award winners
queryString = """
SELECT DISTINCT ?pr ?name 
WHERE { 
?a ?pr wd:Q7146100.

?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('pr', 'http://www.wikidata.org/prop/direct/P2517'), ('name', 'category for recipients of this award')]


2

In [54]:
#see some properties of the category Best Directing Academy Award winners
queryString = """
SELECT DISTINCT ?pr ?name 
WHERE { 
wd:Q7146100 ?pr ?a.

?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('pr', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('pr', 'http://www.wikidata.org/prop/direct/P4224'), ('name', 'category contains')]
[('pr', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]


5

Categories tell me nothing. Look again to the academy awards property

In [79]:
#properties of academy awards
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a ?pr wd:Q19020.

?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('pr', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('pr', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('pr', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('pr', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('pr', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('pr', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('pr', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')]
[('pr', 'http://www.wikidata.

15

Look at `wdt:P179` part of the series

In [80]:
#properties of edition of academy awards
queryString = """
SELECT DISTINCT ?a ?name
WHERE { 
?a wdt:P179 wd:Q19020.

?a sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q22076019'), ('name', '89th Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q148609'), ('name', '16th Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q918125'), ('name', '45th Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q1065282'), ('name', '51st Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q543438'), ('name', '52nd Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q938235'), ('name', '58th Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q24636843'), ('name', '90th Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q508528'), ('name', '15th Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q508554'), ('name', '13th Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q257443'), ('name', '60th Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q910864'), ('name', '62nd Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q917138'), ('name', '37th Academy A

86

These represents the academy awards in the years, look to the class of these istances and then look at all the properties

In [81]:
#istances
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
?a wdt:P179 wd:Q19020.
?a wdt:P31 ?x.
?x sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q15416'), ('name', 'television program')]
[('x', 'http://www.wikidata.org/entity/Q15275719'), ('name', 'recurring event')]
[('x', 'http://www.wikidata.org/entity/Q16913666'), ('name', 'Academy Awards ceremony')]


3

In [82]:
#properties of edition of academy awards
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a wdt:P179 wd:Q19020.
?a ?pr ?x.
?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('pr', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('pr', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('pr', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('pr', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('pr', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('pr', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('pr', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Kno

36

In [126]:
#properties of edition of academy awards
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a wdt:P179 wd:Q19020.
?x ?pr ?a.
?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('pr', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('pr', 'http://www.wikidata.org/prop/direct/P793'), ('name', 'significant event')]
[('pr', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
[('pr', 'http://www.wikidata.org/prop/direct/P1855'), ('name', 'Wikidata property example')]


8

In [88]:
#properties of edition of academy awards
queryString = """
SELECT DISTINCT ?name (count(*) AS ?cont)
WHERE { 
?a wdt:P179 wd:Q19020.
?x wdt:P1411 ?a.
?x sc:name ?name .
} 
GROUP BY ?name
ORDER BY DESC (?cont)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('name', 'Valentyn Vasyanovych'), ('cont', '1')]


1

In [89]:
#properties of edition of academy awards
queryString = """
SELECT DISTINCT ?name (count(*) AS ?cont)
WHERE { 
?a wdt:P179 wd:Q19020.
?x wdt:P166 ?a.
?x sc:name ?name .
} 
GROUP BY ?name
ORDER BY DESC (?cont)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('name', 'Geoffrey Unsworth'), ('cont', '2')]
[('name', 'Arthur Charles Miller'), ('cont', '2')]
[('name', 'Haskell Wexler'), ('cont', '2')]
[('name', 'Christopher McQuarrie'), ('cont', '1')]
[('name', 'Luis Bacalov'), ('cont', '1')]
[('name', 'Viktor Komar'), ('cont', '1')]
[('name', 'W. Howard Greene'), ('cont', '1')]
[('name', 'Sam Smith'), ('cont', '1')]
[('name', 'Eugen Schüfftan'), ('cont', '1')]
[('name', 'James L. Brooks'), ('cont', '1')]
[('name', 'Susanne Bier'), ('cont', '1')]
[('name', 'Cameron Crowe'), ('cont', '1')]
[('name', 'Ghislain Cloquet'), ('cont', '1')]
[('name', 'Yul Brynner'), ('cont', '1')]
[('name', 'Karl Struss'), ('cont', '1')]
[('name', 'Moshé Mizrahi'), ('cont', '1')]
[('name', 'Robert E. Sherwood'), ('cont', '1')]
[('name', 'Joseph A. Valentine'), ('cont', '1')]
[('name', 'Jennifer Lawrence'), ('cont', '1')]
[('name', 'Roger Deakins'), ('cont', '1')]
[('name', 'Ted Moore'), ('cont', '1')]


21

Found Academy awards ceremony `wd:Q16913666`. Look how a director for example Woody Allen is related with these istances.

In [84]:
#properties of edition of academy awards
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a wdt:P31 wd:Q16913666.
?a ?pr ?dir.
?dir wdt:P106 wd:Q3455803.
?pr sc:name ?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P371'), ('name', 'presenter')]


1

In [85]:
#properties of edition of academy awards
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a wdt:P31 wd:Q16913666.
?dir ?pr ?a.
?dir wdt:P106 wd:Q3455803.
?pr sc:name ?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

Zero connection. Try to verify if there are something in the middle

In [90]:
queryString = """
SELECT DISTINCT ?a ?name
WHERE { 
?a wdt:P31 wd:Q16913666.
?dir wdt:P106 wd:Q3455803.
?b ?z ?a.
?dir ?pr ?b.
?a sc:name ?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [91]:
queryString = """
SELECT DISTINCT ?b ?name
WHERE { 
?a wdt:P31 wd:Q16913666.
?dir wdt:P106 wd:Q3455803.
?a ?z ?b.
?dir ?pr ?b.
?b sc:name ?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('b', 'http://www.wikidata.org/entity/Q1860'), ('name', 'English')]
[('b', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q95030'), ('name', 'Warren Beatty')]
[('b', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('b', 'http://www.wikidata.org/entity/Q65'), ('name', 'Los Angeles')]
[('b', 'http://www.wikidata.org/entity/Q99'), ('name', 'California')]
[('b', 'http://www.wikidata.org/entity/Q47164'), ('name', 'Santa Monica')]
[('b', 'http://www.wikidata.org/entity/Q294912'), ('name', 'Richard Pryor')]
[('b', 'http://www.wikidata.org/entity/Q34006'), ('name', 'Hollywood')]
[('b', 'http://www.wikidata.org/entity/Q108366'), ('name', 'Gregory Peck')]
[('b', 'http://www.wikidata.org/entity/Q643106'), ('name', 'Alan King')]


11

In [92]:
queryString = """
SELECT DISTINCT ?b ?name
WHERE { 
?a wdt:P31 wd:Q16913666.
?dir wdt:P106 wd:Q3455803.
?a ?z ?b.
?b ?pr ?dir .
?b sc:name ?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('b', 'http://www.wikidata.org/entity/Q95030'), ('name', 'Warren Beatty')]
[('b', 'http://www.wikidata.org/entity/Q83338'), ('name', 'Robin Williams')]
[('b', 'http://www.wikidata.org/entity/Q910864'), ('name', '62nd Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q108366'), ('name', 'Gregory Peck')]
[('b', 'http://www.wikidata.org/entity/Q222461'), ('name', '65th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q857047'), ('name', '64th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q388978'), ('name', '48th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q6550'), ('name', 'Donald Duck')]
[('b', 'http://www.wikidata.org/entity/Q178975'), ('name', '31st Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q536749'), ('name', '70th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q903336'), ('name', '69th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q910856'), ('name', '63rd Academy Awards')]
[('b', 'http://www.wikidat

16

In [93]:
queryString = """
SELECT DISTINCT ?b ?name
WHERE { 
?a wdt:P31 wd:Q16913666.
?dir wdt:P106 wd:Q3455803.
?b ?z ?a.
?b ?pr ?dir .
?b sc:name ?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('b', 'http://www.wikidata.org/entity/Q910864'), ('name', '62nd Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q388978'), ('name', '48th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q222461'), ('name', '65th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q857047'), ('name', '64th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q178975'), ('name', '31st Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q536749'), ('name', '70th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q903336'), ('name', '69th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q910856'), ('name', '63rd Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q180675'), ('name', '84th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q458646'), ('name', '76th Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q429319'), ('name', '72nd Academy Awards')]
[('b', 'http://www.wikidata.org/entity/Q917235'), ('name', '59th Academy Awards

12

In [94]:
queryString = """
SELECT DISTINCT ?z1 ?pr1
WHERE { 
?a wdt:P31 wd:Q16913666.
?dir wdt:P106 wd:Q3455803.
?b ?z ?a.
?b ?pr ?dir .
?z sc:name ?z1 .
?pr sc:name ?pr1 .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('z1', 'follows'), ('pr1', 'presenter')]
[('z1', 'followed by'), ('pr1', 'presenter')]


2

Nothing interest but I found that some directors were also a presenter to the Academy Award ceremony. Try to use wdt:P3342 (significant person) given in the head of the workflow

In [103]:
queryString = """
SELECT DISTINCT ?a ?name
WHERE { 
?dir wdt:P106 wd:Q3455803.
?dir wdt:P3342 ?a.
OPTIONAL{?a sc:name ?name .}
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q104822989'), ('name', 'Luigi Benzi')]
[('a', 'http://www.wikidata.org/entity/Q964609'), ('name', 'Sonja Wigert')]
[('a', 'http://www.wikidata.org/entity/Q1778583'), ('name', 'Aase Nordmo Løvberg')]
[('a', 'http://www.wikidata.org/entity/Q4988547'), ('name', 'Ida Gawell-Blumenthal')]
[('a', 'http://www.wikidata.org/entity/Q89137'), ('name', 'Ute Bock')]
[('a', 'http://www.wikidata.org/entity/Q106893140'), ('name', 'Mina')]
[('a', 'http://www.wikidata.org/entity/Q777997'), ('name', 'Arseny Tarkovsky')]
[('a', 'http://www.wikidata.org/entity/Q99270809'), ('name', 'Mykola Stanislavskyi')]
[('a', 'http://www.wikidata.org/entity/Q95349717'), ('name', 'Harry Sachs')]
[('a', 'http://www.wikidata.org/entity/Q95349718'), ('name', 'Franz Hoefner')]


10

In [105]:
queryString = """
SELECT DISTINCT ?a ?name
WHERE { 
?dir wdt:P106 wd:Q3455803.
?a wdt:P3342 ?dir.
OPTIONAL{?a sc:name ?name .}
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q958683'), ('name', 'Nick Kamen')]
[('a', 'http://www.wikidata.org/entity/Q243068'), ('name', 'Anne Charlotte Leffler')]
[('a', 'http://www.wikidata.org/entity/Q1611548'), ('name', 'Hermann Huber')]
[('a', 'http://www.wikidata.org/entity/Q43247'), ('name', 'Ingrid Bergman')]
[('a', 'http://www.wikidata.org/entity/Q104822989'), ('name', 'Luigi Benzi')]
[('a', 'http://www.wikidata.org/entity/Q964609'), ('name', 'Sonja Wigert')]
[('a', 'http://www.wikidata.org/entity/Q6905657'), ('name', 'Montgomery Blair High School')]
[('a', 'http://www.wikidata.org/entity/Q915034'), ('name', 'Mimi Pollak')]
[('a', 'http://www.wikidata.org/entity/Q8028532'), ('name', 'Witt/Thomas Productions')]
[('a', 'http://www.wikidata.org/entity/Q5151496'), ('name', 'comedy of menace')]
[('a', 'http://www.wikidata.org/entity/Q98665900'), ('name', 'Ferdinand Reyher')]
[('a', 'http://www.wikidata.org/entity/Q1778583'), ('name', 'Aase Nordmo Løvberg')]
[('a', 'http://www.w

21

Last try: start from director and look again if they are connected with something related to awards

In [9]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 
?dir wdt:P106 wd:Q3455803;
    ?pr ?x.
OPTIONAL{?x sc:name ?name .}
OPTIONAL{?pr sc:name ?prop .}
FILTER(REGEX(?name,".*ward.*"))
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('prop', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('prop', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P735'), ('prop', 'given name')]
[('pr', 'http://www.wikidata.org/prop/direct/P737'), ('prop', 'influenced by')]
[('pr', 'http://www.wikidata.org/prop/direct/P2354'), ('prop', 'has list')]
[('pr', 'http://www.wikidata.org/prop/direct/P26'), ('prop', 'spouse')]
[('pr', 'http://www.wikidata.org/prop/direct/P69'), ('prop', 'educated at')]
[('pr', 'http://www.wikidata.org/prop/direct/P7084'), ('prop', 'related category')]
[('pr', 'http://www.wikidata.org/prop/direct/P910'), ('prop', "topic's main category")]
[('pr', 'http://www.wikidata.org/prop/direct/P3373'), ('prop', 'sibling')]
[('pr', 'http://www.wikidata.org/prop/direct/P734'), ('prop', 'family name')]
[('pr', 'http://www.wikidata.org/prop/direct/P1889'), ('prop', 'different from')]
[('pr', 'http://www.wikidata.org/prop/dir

16

In [10]:
# use the property participant and look which are the objects
queryString = """
SELECT DISTINCT ?ist ?ista
WHERE { 
?dir wdt:P106 wd:Q3455803;
    wdt:P1344 ?x.
?x wdt:P31 ?ist.
?ist sc:name ?ista.
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('ist', 'http://www.wikidata.org/entity/Q82414'), ('ista', 'Winter Olympic Games')]
[('ist', 'http://www.wikidata.org/entity/Q667276'), ('ista', 'art exhibition')]
[('ist', 'http://www.wikidata.org/entity/Q72823'), ('ista', 'documenta')]
[('ist', 'http://www.wikidata.org/entity/Q15416'), ('ista', 'television program')]
[('ist', 'http://www.wikidata.org/entity/Q1955280'), ('ista', 'music competition')]
[('ist', 'http://www.wikidata.org/entity/Q868557'), ('ista', 'music festival')]
[('ist', 'http://www.wikidata.org/entity/Q15275719'), ('ista', 'recurring event')]
[('ist', 'http://www.wikidata.org/entity/Q1850936'), ('ista', 'benefit concert')]
[('ist', 'http://www.wikidata.org/entity/Q182832'), ('ista', 'concert')]
[('ist', 'http://www.wikidata.org/entity/Q5398426'), ('ista', 'television series')]
[('ist', 'http://www.wikidata.org/entity/Q103495'), ('ista', 'world war')]
[('ist', 'http://www.wikidata.org/entity/Q11514315'), ('ista', 'historical period')]
[('ist', 'http://www.wik

40

In [14]:
#Q4504495 award ceremony
queryString = """
SELECT DISTINCT ?x ?ista
WHERE { 
?dir wdt:P106 wd:Q3455803;
    wdt:P1344 ?x.
?x wdt:P31 wd:Q4504495.
?x sc:name ?ista.
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q13561521'), ('ista', 'Ingeborg Bachmann Award 1991')]
[('x', 'http://www.wikidata.org/entity/Q1662846'), ('ista', 'Ingeborg Bachmann Award 1980')]


2

In [13]:
# try to yse the property has list
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
?dir wdt:P106 wd:Q3455803;
    wdt:P2354 ?x.
OPTIONAL{?x sc:name ?name.}
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q48998364'), ('name', 'list of awards and nominations received by Dakota Johnson')]
[('x', 'http://www.wikidata.org/entity/Q77034675')]


2

In [15]:
# try to yse the property has list
queryString = """
SELECT DISTINCT ?ist ?name
WHERE { 
?dir wdt:P106 wd:Q3455803;
    wdt:P1411 ?x.
?x wdt:P31 ?ist.
OPTIONAL{?ist sc:name ?name.}
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('ist', 'http://www.wikidata.org/entity/Q682154'), ('name', 'Drama Desk Award')]
[('ist', 'http://www.wikidata.org/entity/Q724384'), ('name', 'Magnesia Litera')]
[('ist', 'http://www.wikidata.org/entity/Q96474688'), ('name', 'award for best film by genre')]
[('ist', 'http://www.wikidata.org/entity/Q96474679'), ('name', 'award for best direction')]
[('ist', 'http://www.wikidata.org/entity/Q96474603'), ('name', 'award for best supporting actor')]
[('ist', 'http://www.wikidata.org/entity/Q56116950'), ('name', 'film award category')]
[('ist', 'http://www.wikidata.org/entity/Q21866939'), ('name', 'National Society of Film Critics Award')]
[('ist', 'http://www.wikidata.org/entity/Q1011547'), ('name', 'Golden Globe Award')]
[('ist', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('ist', 'http://www.wikidata.org/entity/Q96474687'), ('name', 'award for best film')]
[('ist', 'http://www.wikidata.org/entity/Q15229229'), ('name', 'theatre award')]
[('ist', 'http://

89

Conclusion: I tried to explore a lot of things but I don't find the information about the single Academy Award. 

I perform the last query of this notebook showing the academy awards won by the directors also counting the awards receive by the film that they directed.

In [19]:
#find how films are connected with academy awards
queryString = """
SELECT DISTINCT ?director ?film (COUNT( ?a) AS ?cont)
WHERE { 
?f wdt:P31 wd:Q11424.
?a wdt:P31 wd:Q19020.
?f wdt:P166 ?a.
?f wdt:P57 ?dir.

?f sc:name ?film .
?dir sc:name ?director .
} 
GROUP BY ?f ?film ?dir ?director
ORDER BY DESC (?cont)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('director', 'Peter Jackson'), ('film', 'The Lord of the Rings: The Return of the King'), ('cont', '11')]
[('director', 'William Wyler'), ('film', 'Ben-Hur'), ('cont', '11')]
[('director', 'James Cameron'), ('film', 'Titanic'), ('cont', '11')]
[('director', 'Robert Wise'), ('film', 'West Side Story'), ('cont', '10')]
[('director', 'Jerome Robbins'), ('film', 'West Side Story'), ('cont', '10')]
[('director', 'Bernardo Bertolucci'), ('film', 'The Last Emperor'), ('cont', '9')]
[('director', 'Anthony Minghella'), ('film', 'The English Patient'), ('cont', '9')]
[('director', 'Vincente Minnelli'), ('film', 'Gigi'), ('cont', '8')]
[('director', 'Elia Kazan'), ('film', 'On the Waterfront'), ('cont', '8')]
[('director', 'Bob Fosse'), ('film', 'Cabaret'), ('cont', '8')]


10

In [20]:
#show the sum of the academy awards winning by the film grouping by they director
queryString = """
SELECT DISTINCT ?director (COUNT( ?a) AS ?cont)
WHERE { 
?f wdt:P31 wd:Q11424.
?a wdt:P31 wd:Q19020.
?f wdt:P166 ?a.
?f wdt:P57 ?dir.

?f sc:name ?film .
?dir sc:name ?director .
} 
GROUP BY ?dir ?director
ORDER BY DESC (?cont)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('director', 'William Wyler'), ('cont', '41')]
[('director', 'Steven Spielberg'), ('cont', '32')]
[('director', 'George Cukor'), ('cont', '30')]
[('director', 'David Lean'), ('cont', '30')]
[('director', 'Fred Zinnemann'), ('cont', '23')]
[('director', 'John Ford'), ('cont', '21')]
[('director', 'Elia Kazan'), ('cont', '21')]
[('director', 'Vincente Minnelli'), ('cont', '21')]
[('director', 'Martin Scorsese'), ('cont', '20')]
[('director', 'Peter Jackson'), ('cont', '20')]


10

In [22]:
#show the sum of the academy awards nomination by the film grouping by they director
queryString = """
SELECT DISTINCT ?director (COUNT( ?a) AS ?cont)
WHERE { 
?f wdt:P31 wd:Q11424.
?a wdt:P31 wd:Q19020.
?f wdt:P1411 ?a.
?f wdt:P57 ?dir.

?f sc:name ?film .
?dir sc:name ?director .
} 
GROUP BY ?dir ?director
ORDER BY DESC (?cont)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('director', 'William Wyler'), ('cont', '127')]
[('director', 'Steven Spielberg'), ('cont', '122')]
[('director', 'George Cukor'), ('cont', '92')]
[('director', 'Martin Scorsese'), ('cont', '80')]
[('director', 'John Ford'), ('cont', '78')]
[('director', 'Michael Curtiz'), ('cont', '75')]
[('director', 'Billy Wilder'), ('cont', '73')]
[('director', 'George Stevens'), ('cont', '63')]
[('director', 'Fred Zinnemann'), ('cont', '63')]
[('director', 'Henry King'), ('cont', '63')]


10

In [24]:
#academy award nominations by directors
queryString = """
SELECT DISTINCT ?director ?nominations ?wins
WHERE { 
{
    SELECT DISTINCT ?x1 (COUNT(*) AS ?nominations)
    WHERE { 
    ?f1 wdt:P31 wd:Q11424.
    ?a1 wdt:P31 wd:Q19020.
    ?f1 wdt:P1411 ?a1.
    ?f1 wdt:P57 ?x1.
    } 
    GROUP BY ?x1 
}
{
    SELECT DISTINCT ?x2 (COUNT(*) AS ?wins)
    WHERE { 
    ?f2 wdt:P31 wd:Q11424.
    ?a2 wdt:P31 wd:Q19020.
    ?f2 wdt:P166 ?a2.
    ?f2 wdt:P57 ?x2.

    } 
    GROUP BY ?x2
}
FILTER(?x1=?x2)
?x1 sc:name ?director
} 
ORDER BY DESC ( ?nominations) DESC (?wins)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('director', 'William Wyler'), ('nominations', '127'), ('wins', '41')]
[('director', 'Steven Spielberg'), ('nominations', '122'), ('wins', '32')]
[('director', 'George Cukor'), ('nominations', '92'), ('wins', '30')]
[('director', 'Martin Scorsese'), ('nominations', '80'), ('wins', '20')]
[('director', 'John Ford'), ('nominations', '78'), ('wins', '21')]
[('director', 'Michael Curtiz'), ('nominations', '75'), ('wins', '15')]
[('director', 'Billy Wilder'), ('nominations', '73'), ('wins', '17')]
[('director', 'David Lean'), ('nominations', '63'), ('wins', '30')]
[('director', 'Fred Zinnemann'), ('nominations', '63'), ('wins', '23')]
[('director', 'Henry King'), ('nominations', '63'), ('wins', '16')]
[('director', 'George Stevens'), ('nominations', '63'), ('wins', '13')]
[('director', 'Robert Wise'), ('nominations', '62'), ('wins', '18')]
[('director', 'Sam Wood'), ('nominations', '61'), ('wins', '13')]
[('director', 'Mervyn LeRoy'), ('nominations', '60'), ('wins', '10')]
[('direc

20